In [131]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [132]:
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv')

In [133]:
df.head()

lead_source    industry  number_of_courses_viewed  annual_income  \
0      paid_ads         NaN                         1        79450.0   
1  social_media      retail                         1        46992.0   
2        events  healthcare                         5        78796.0   
3      paid_ads      retail                         2        83843.0   
4      referral   education                         3        85012.0   

  employment_status       location  interaction_count  lead_score  converted  
0        unemployed  south_america                  4        0.94          1  
1          employed  south_america                  1        0.80          0  
2        unemployed      australia                  3        0.69          1  
3               NaN      australia                  1        0.87          0  
4     self_employed         europe                  3        0.62          1

### Data Preparation

In [134]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [135]:
categorical = list(df.dtypes[df.dtypes == 'object'].index)

In [136]:
categorical

['lead_source', 'industry', 'employment_status', 'location']

In [137]:
df[categorical] = df[categorical].fillna('NA')
df['annual_income'] = df['annual_income'].fillna(0.0)

In [138]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [139]:
df.head()

lead_source    industry  number_of_courses_viewed  annual_income  \
0      paid_ads          NA                         1        79450.0   
1  social_media      retail                         1        46992.0   
2        events  healthcare                         5        78796.0   
3      paid_ads      retail                         2        83843.0   
4      referral   education                         3        85012.0   

  employment_status       location  interaction_count  lead_score  converted  
0        unemployed  south_america                  4        0.94          1  
1          employed  south_america                  1        0.80          0  
2        unemployed      australia                  3        0.69          1  
3                NA      australia                  1        0.87          0  
4     self_employed         europe                  3        0.62          1

## Question 1
### What is the most frequent observation (mode) for the column industry?

In [140]:
df['industry'].mode()

0    retail
Name: industry, dtype: object

### Answer : retail

## Question 2
### Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.

### What are the two features that have the biggest correlation?

In [141]:
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']

In [142]:
df[numerical].corr()

number_of_courses_viewed  annual_income  \
number_of_courses_viewed                  1.000000       0.009770   
annual_income                             0.009770       1.000000   
interaction_count                        -0.023565       0.027036   
lead_score                               -0.004879       0.015610   

                          interaction_count  lead_score  
number_of_courses_viewed          -0.023565   -0.004879  
annual_income                      0.027036    0.015610  
interaction_count                  1.000000    0.009888  
lead_score                         0.009888    1.000000

### Answer : annual_income, interaction_count

### Split the data
#### Split your data in train/val/test sets with 60%/20%/20% distribution.
#### Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
#### Make sure that the target value y is not in your dataframe.

In [143]:
from sklearn.model_selection import train_test_split

In [144]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [145]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [146]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [147]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_train['converted']
del df_val['converted']
del df_test['converted']

## Question 3
### Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only.
#### Round the scores to 2 decimals using round(score, 2).

In [148]:
from sklearn.metrics import mutual_info_score

In [149]:
def mutual_info_converted_score(series):
    return mutual_info_score(series, y_train)

In [150]:
mi = df_train[categorical].apply(mutual_info_converted_score)
round(mi.sort_values(ascending=False),2)


lead_source          0.04
employment_status    0.01
industry             0.01
location             0.00
dtype: float64

### Answer : lead_score

## Question 4
### Now let's train a logistic regression.
#### Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
#### Fit the model on the training dataset.
#### To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
#### model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
#### Calculate the accuracy on the validation dataset and round it to 2 decimal digits.
### What accuracy did you get?

In [151]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [152]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [153]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [154]:
y_pred = model.predict_proba(X_val)[:, 1]

In [155]:
converted_decision = (y_pred >= 0.5)

In [156]:
(y_val == converted_decision).mean()

np.float64(0.6996587030716723)

In [157]:
df_pred = pd.DataFrame()
df_pred['probability'] = y_pred
df_pred['prediction'] = converted_decision.astype(int)
df_pred['actual'] = y_val

In [158]:
df_pred['correct'] = df_pred.prediction == df_pred.actual

In [159]:
df_pred.correct.mean().round(2)

np.float64(0.7)

### Answer : 0.74 (closest to the options)

## Question 5
#### Let's find the least useful feature using the feature elimination technique.
#### Train a model using the same features and parameters as in Q4 (without rounding).
#### Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
#### For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
### Which of following feature has the smallest difference?

##### Without feature 'industry'

In [160]:
df_train.head()

lead_source       industry  number_of_courses_viewed  annual_income  \
0        paid_ads         retail                         0        58472.0   
1  organic_search  manufacturing                         3        71738.0   
2        paid_ads     technology                         3        81973.0   
3              NA     technology                         1        74956.0   
4  organic_search         retail                         3        59335.0   

  employment_status       location  interaction_count  lead_score  
0           student    middle_east                  5        0.03  
1           student    middle_east                  6        0.77  
2          employed  north_america                  2        0.59  
3          employed         europe                  3        0.34  
4           student      australia                  1        0.98

In [161]:
categorical_noindustry = ['lead_source', 'employment_status', 'location']

In [162]:
train_dict_ni = df_train[categorical_noindustry + numerical].to_dict(orient='records')
X_train_ni = dv.fit_transform(train_dict_ni)

val_dict_ni = df_val[categorical_noindustry + numerical].to_dict(orient='records')
X_val_ni = dv.transform(val_dict_ni)

In [163]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train_ni, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [165]:
y_pred = model.predict_proba(X_val_ni)[:, 1]
converted_decision = (y_pred >= 0.5)
(y_val == converted_decision).mean()

np.float64(0.6996587030716723)

In [167]:
df_pred = pd.DataFrame()
df_pred['probability'] = y_pred
df_pred['prediction'] = converted_decision.astype(int)
df_pred['actual'] = y_val
df_pred['correct'] = df_pred.prediction == df_pred.actual
df_pred.correct.mean()

np.float64(0.6996587030716723)

##### Without feature 'employment_status'

In [168]:
categorical_no_es = ['lead_source', 'industry', 'location']

In [169]:
train_dict_no_es = df_train[categorical_no_es + numerical].to_dict(orient='records')
X_train_no_es = dv.fit_transform(train_dict_no_es)

val_dict_no_es = df_val[categorical_no_es + numerical].to_dict(orient='records')
X_val_no_es = dv.transform(val_dict_no_es)

In [170]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train_no_es, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [171]:
y_pred = model.predict_proba(X_val_no_es)[:, 1]
converted_decision = (y_pred >= 0.5)
(y_val == converted_decision).mean()

np.float64(0.6962457337883959)

##### Without feature 'lead_score'

In [172]:
categorical_no_ls = ['industry', 'employment_status', 'location']

In [173]:
train_dict_no_ls = df_train[categorical_no_ls + numerical].to_dict(orient='records')
X_train_no_ls = dv.fit_transform(train_dict_no_ls)

val_dict_no_ls = df_val[categorical_no_ls + numerical].to_dict(orient='records')
X_val_no_ls = dv.transform(val_dict_no_ls)

In [174]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train_no_ls, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [175]:
y_pred = model.predict_proba(X_val_no_ls)[:, 1]
converted_decision = (y_pred >= 0.5)
(y_val == converted_decision).mean()

np.float64(0.7030716723549488)

##### Original Accuracy = 0.6996587030716723
##### Accuracy without 'Industry' = 0.6996587030716723
##### Accuracy without 'employment_status' = 0.6962457337883959
##### Accuracy without 'lead_source' = 0.7030716723549488

In [176]:
0.6996587030716723 - 0.6996587030716723

0.0

In [177]:
0.6996587030716723 - 0.6962457337883959

0.0034129692832763903

In [178]:
0.6996587030716723 - 0.7030716723549488

-0.0034129692832765013

### Answer : lead_score

## Question 6
#### Now let's train a regularized logistic regression.
#### Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
#### Train models using all the features as in Q4.
#### Calculate the accuracy on the validation dataset and round it to 3 decimal digits.
### Which of these C leads to the best accuracy on the validation set?

In [179]:
categorical = ['lead_source', 'industry', 'employment_status', 'location']
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']

In [180]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [185]:
C_values = [0.01, 0.1, 1, 10, 100]
for C in C_values:
    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    converted_decision = (y_pred >= 0.5)
    accuracy = (y_val == converted_decision).mean()
    print(f"C = {C:<7} → Accuracy = {accuracy:.3f}")

C = 0.01    → Accuracy = 0.700
C = 0.1     → Accuracy = 0.700
C = 1       → Accuracy = 0.700
C = 10      → Accuracy = 0.700
C = 100     → Accuracy = 0.700


### Answer : C = 0.01